<a href="https://colab.research.google.com/github/tiwari-arpit/nlp/blob/main/SeNER_Hindi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModel, AutoConfig, AutoModelForTokenClassification, AutoTokenizer

In [ ]:
model_name = "arpit-tiwari/distilbert-finetuned-hindi-ner"
model_checkpoint = AutoModelForTokenClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
from datasets import load_dataset
lang = 'hi'
data = load_dataset('ai4bharat/naamapadam',lang)

In [ ]:
hindi_data = data
hindi_data['train'] = data['train'].select(range(50000,100001))

In [ ]:
tags = hindi_data['train'].features['ner_tags'].feature
def create_tag_name(batch):
  tag_name = {'ner_tags_str': [ tags.int2str(idx) for idx in batch['ner_tags']]}
  return tag_name

hindi_data = hindi_data.map(create_tag_name)

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_dataset = hindi_data.map(tokenize_and_align_labels,batched=True,remove_columns=hindi_data['train'].column_names)

In [ ]:
class ArrowAttention(nn.Module):
    def __init__(self, hidden_size, window_size, num_heads=8):
        super().__init__()
        self.window_size = window_size
        self.num_heads = num_heads
        self.head_dim = hidden_size // num_heads
        self.cls_attention = nn.MultiheadAttention(hidden_size, num_heads)
        self.local_attention = nn.MultiheadAttention(hidden_size, num_heads)

    def forward(self, hidden_states):
        batch_size, seq_len, hidden_size = hidden_states.size()

        cls_token = hidden_states[:, 0:1, :]
        cls_token = cls_token.transpose(0, 1)
        global_output, _ = self.cls_attention(
            cls_token,
            hidden_states.transpose(0, 1),
            hidden_states.transpose(0, 1)
        )
        global_output = global_output.transpose(0, 1)

        if seq_len > 1:
            local_outputs = []
            for i in range(1, seq_len):
                start = max(1, i - self.window_size)
                end = min(seq_len, i + self.window_size + 1)

                window = hidden_states[:, start:end, :]
                current_token = hidden_states[:, i:i+1, :]

                current_token = current_token.transpose(0, 1)
                window = window.transpose(0, 1)

                local_output, _ = self.local_attention(
                    current_token,
                    window,
                    window
                )
                local_output = local_output.transpose(0, 1)
                local_outputs.append(local_output)

            if local_outputs:
                local_output = torch.cat(local_outputs, dim=1)
                output = torch.cat([global_output, local_output], dim=1)
            else:
                output = global_output
        else:
            output = global_output

        return output

In [ ]:
class LogNScaling(nn.Module):
    def __init__(self, hidden_size):
        super(LogNScaling, self).__init__()
        self.scale = nn.Parameter(torch.tensor(1.0 / (hidden_size ** 0.5)))

    def forward(self, attention_scores):
        return attention_scores * self.scale

In [ ]:
class BiSPA(nn.Module):
    def __init__(self, hidden_size, window_size, num_heads=8):
        super(BiSPA, self).__init__()
        self.window_size = window_size
        self.num_heads = num_heads
        self.horizontal_attention = nn.MultiheadAttention(hidden_size, num_heads)
        self.vertical_attention = nn.MultiheadAttention(hidden_size, num_heads)
        self.mlp = nn.Sequential(
            nn.Linear(2 * hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size)
        )

    def forward(self, hidden_states):
        batch_size, seq_len, hidden_size = hidden_states.size()
        hidden_states_t = hidden_states.transpose(0, 1)

        # Horizontal attention
        horizontal_outputs = []
        for i in range(seq_len):
            start = max(0, i - self.window_size)
            end = min(seq_len, i + self.window_size + 1)
            horizontal_input = hidden_states[start:end, :, :]
            query = hidden_states[i:i+1, :, :]

            horizontal_output, _ = self.horizontal_attention(
                query,
                horizontal_input,
                horizontal_input
            )
            horizontal_outputs.append(horizontal_output)

        horizontal_output = torch.cat(horizontal_outputs, dim=0)

        # Vertical attention
        vertical_outputs = []
        for i in range(seq_len):
            start = max(0, i - self.window_size)
            end = min(seq_len, i + self.window_size + 1)
            query = hidden_states_t[i:i+1, :, :]
            vertical_input = hidden_states[start:end, :, :]
            vertical_output, _ = self.vertical_attention(
                query,
                vertical_input,
                vertical_input
            )
            vertical_outputs.append(vertical_output)

        vertical_output = torch.cat(vertical_outputs, dim=0)

        # Combine outputs
        combined_output = torch.cat([horizontal_output, vertical_output], dim=-1)
        output = self.mlp(combined_output)
        output = output.transpose(0, 1)

        return output

In [ ]:
class SeNER(nn.Module):
    def __init__(self, model_name, num_labels, window_size=128):
        super(SeNER, self).__init__()
        self.config = AutoConfig.from_pretrained(model_name)
        self.base_model = AutoModel.from_pretrained(model_name, config=self.config)
        self.num_labels = num_labels
        self.arrow_attention = ArrowAttention(self.config.hidden_size, window_size)
        self.log_n_scaling = LogNScaling(self.config.hidden_size)
        self.bispa = BiSPA(self.config.hidden_size, window_size)
        self.classifier = nn.Linear(self.config.hidden_size, num_labels)
        self.dropout = nn.Dropout(self.config.hidden_size_dropout if hasattr(self.config, 'hidden_size_dropout') else 0.1)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state

        attended_output = self.arrow_attention(sequence_output)

        if attended_output.size(1) > 0:
            cls_token = attended_output[:, 0:1, :]
            scaled_cls = self.log_n_scaling(cls_token)

            if attended_output.size(1) > 1:
                attended_output = torch.cat([scaled_cls, attended_output[:, 1:, :]], dim=1)
            else:
                attended_output = scaled_cls

        sequence_output = self.bispa(attended_output)
        sequence_output = self.dropout(sequence_output)
        logits = self.classifier(sequence_output)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            active_loss = labels.view(-1) != -100
            active_logits = logits.view(-1, self.num_labels)[active_loss]
            active_labels = labels.view(-1)[active_loss]
            loss = loss_fct(active_logits, active_labels)

        return {'loss': loss, 'logits': logits} if loss is not None else {'logits': logits}


In [ ]:
from transformers import DataCollatorForTokenClassification
data_colator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import os

# if not os.path.exists(extract_dir):
#     os.makedirs(extract_dir)

with zipfile.ZipFile("/content/MyDrive/MyDrive/SeNER.zip", 'r') as zip_ref:
    zip_ref.extractall("extract_dir")

os.listdir("/content/extract_dir/")

['content']

In [ ]:
import os
os.listdir("drive/MyDrive/nlp/SeNER/")

['tokenizer_config.json',
 'special_tokens_map.json',
 'vocab.txt',
 'model.safetensors',
 'training_args.bin',
 'tokenizer.json']

In [ ]:
num_labels = len(tags.names)
model = SeNER(model_name, num_labels)

In [ ]:
from safetensors.torch import load_file
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

state_dict = load_file("drive/MyDrive/nlp/SeNER/model.safetensors")
model.load_state_dict(state_dict)
model.to(device)

SeNER(
  (base_model): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in_

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./SeNER_Hindi",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    eval_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    remove_unused_columns=False,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator = data_colator,
)

trainer.train()

In [ ]:
results = trainer.evaluate()
print(results)

In [ ]:
trainer.save_model("./SeNER")

In [ ]:
def predict_entities(model, tokenizer, text, id2label=None):
    inputs = tokenizer(text, return_offsets_mapping=True, return_tensors="pt", truncation=True, padding=True)
    offset_mapping = inputs.pop("offset_mapping")
    inputs = {k: v.to(next(model.parameters()).device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs["logits"]
    predictions = torch.argmax(logits, dim=-1)[0].cpu().numpy()
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    offset_mapping = offset_mapping[0].cpu().numpy()

    word_entities = []
    previous_word = None
    current_word = ""
    current_label = None

    for token, pred, offset in zip(tokens, predictions, offset_mapping):
        if offset[0] == 0 and offset[1] == 0:
            continue

        start, end = offset
        word = text[start:end]

        if token.startswith("##"):
            current_word += word
        else:
            if current_word:
                label_name = id2label[current_label] if id2label else current_label
                word_entities.append((current_word, label_name))
            current_word = word
            current_label = pred

    if current_word:
        label_name = id2label[current_label] if id2label else current_label
        word_entities.append((current_word, label_name))

    return word_entities


In [ ]:
#text = "इटली की पीएम जॉर्जिया मेलोनी ने पूरी दुनिया के लेफ्टिस्ट लीडर्स को पाखंडी बताया है। उन्होंने कहा कि दुनियाभर में मोदी, ट्रम्प और मेरे जैसे दक्षिणपंथी नेताओं के उभरने से सारे लेफ्टिस्ट नेता परेशान हो गए हैं|"
# text = "भारत की नरेंद्र मोदी सरकार अमेरिकी राष्ट्रपति डॉनल्ड ट्रंप द्वारा लगाए गए 27% आयात शुल्क के खिलाफ किसी भी जवाबी कार्रवाई की योजना फिलहाल नहीं बना रही है. अंतरराष्ट्रीय न्यूज़ एजेंसी रॉयटर्स ने एक वरिष्ठ सरकारी अधिकारी के हवाले से लिखा है कि दिल्ली और वॉशिंगटन डीसी के बीच व्यापार समझौते पर बातचीत जारी है, और इसी कारण भारत संयम बरत रहा है. रिपोर्ट के मुताबिक अधिकारी ने बताया कि केंद्र सरकार डॉनल्ड ट्रंप के टैरिफ ऑर्डर में दिए गए उस प्रावधान का ‘अध्ययन’ कर रही है जिसमें कहा गया है कि जो देश व्यापार को बराबरी पर लाने के लिए सार्थक कदम उठाते हैं, उन्हें छूट दी जा सकती है."
text = "अमेरिका की ओर से लगाए गए 104 फ़ीसदी टैरिफ़ पर चीन के राष्ट्रपति शी जिंगपिंग ने अपनी प्रतिक्रिया दी है. \
शी जिनपिंग ने कहा है कि अमेरिका के टैरिफ़ का सामना करने के लिए चीन को अपने पड़ोसी देशों के साथ रिश्ते मज़बूत करने चाहिए.\
नए अमेरिकी टैरिफ़ लागू होने के बाद यह पहली बार है जब शी जिंगपिंग ने टैरिफ़ पर बात की है.\
उन्होंने ये भी कहा कि एशियाई देशों को मिलकर एक बेहतर भविष्य के लिए काम करना होगा. \
चीन के राष्ट्रपति शी जिनपिंग का ये बयान दक्षिण-पूर्व एशियाई देशों के संगठन आसियान की हालिया टिप्पणी के समान है. आसियान ने भी अपने क्षेत्र में आर्थिक सहयोग बढ़ाने की अपील की थी.\
वियतनाम, कंबोडिया और इंडोनेशिया पर अमेरिकी टैरिफ़ का सबसे ज़्यादा प्रभाव पड़ा है."
predictions = predict_entities(model, tokenizer, text)

print("Input text:", text)
print("Predictions:")
for token, pred in predictions:
    print(f"{token}: {tags.int2str(int(pred))}")


Input text: अमेरिका की ओर से लगाए गए 104 फ़ीसदी टैरिफ़ पर चीन के राष्ट्रपति शी जिंगपिंग ने अपनी प्रतिक्रिया दी है. शी जिनपिंग ने कहा है कि अमेरिका के टैरिफ़ का सामना करने के लिए चीन को अपने पड़ोसी देशों के साथ रिश्ते मज़बूत करने चाहिए.नए अमेरिकी टैरिफ़ लागू होने के बाद यह पहली बार है जब शी जिंगपिंग ने टैरिफ़ पर बात की है.उन्होंने ये भी कहा कि एशियाई देशों को मिलकर एक बेहतर भविष्य के लिए काम करना होगा. चीन के राष्ट्रपति शी जिनपिंग का ये बयान दक्षिण-पूर्व एशियाई देशों के संगठन आसियान की हालिया टिप्पणी के समान है. आसियान ने भी अपने क्षेत्र में आर्थिक सहयोग बढ़ाने की अपील की थी.वियतनाम, कंबोडिया और इंडोनेशिया पर अमेरिकी टैरिफ़ का सबसे ज़्यादा प्रभाव पड़ा है.
Predictions:
अमेरिका: B-LOC
की: O
ओर: O
से: O
लगाए: O
गए: O
104: O
फ़ीसदी: O
टैरिफ़: O
पर: O
चीन: O
के: O
राष्ट्रपति: O
शी: B-PER
जिंगपिंग: I-PER
ने: O
अपनी: O
प्रतिक्रिया: O
दी: O
है: O
.: O
शी: B-PER
जिनपिंग: I-PER
ने: O
कहा: O
है: O
कि: O
अमेरिका: O
के: O
टैरिफ़: O
का: O
सामना: O
करने: O
के: O
लिए: O
चीन: B-LOC
को: O
अपने: O
पड़ोसी:

In [ ]:
tags

ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
import gc
gc.collect()

0

In [ ]:
!zip -r SeNER.zip /content/SeNER

  adding: content/SeNER/ (stored 0%)
  adding: content/SeNER/special_tokens_map.json (deflated 80%)
  adding: content/SeNER/training_args.bin (deflated 52%)
  adding: content/SeNER/tokenizer_config.json (deflated 74%)
  adding: content/SeNER/tokenizer.json (deflated 70%)
  adding: content/SeNER/model.safetensors (deflated 8%)
  adding: content/SeNER/vocab.txt (deflated 49%)


In [ ]:
from google.colab import files
files.download('SeNER.zip')